In [10]:
; Test the LLM client
; Run with: cert.pl test-llm
; 
; Make sure LM Studio is running on http://host.docker.internal:1234
(in-package "ACL2")

 "ACL2"


In [11]:
;; Load the LLM client
(include-book "llm-client" 
              :ttags ((:quicklisp) (:quicklisp.dexador) (:http-json) (:llm-client)))


The event ( INCLUDE-BOOK "llm-client" ...) is redundant.  See :DOC
redundant-events.

Summary
Form:  ( INCLUDE-BOOK "llm-client" ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [12]:
(make-event
 (mv-let (err models state)
   (llm-list-models state)
   (declare (ignorable state))
   (prog2$ 
    (if err
        (cw "~%LIST MODELS ERROR: ~x0~%" err)
      (cw "~%AVAILABLE MODEL IDs (OpenAI compat):~%~x0~%" models))
    (mv nil '(value-triple :list-ok) state))))


AVAILABLE MODEL IDs (OpenAI compat):
("qwen/qwen3-coder-30b" "nvidia/nemotron-3-nano"
                        "s3dev-ai/text-embedding-nomic-embed-text-v1.5"
                        "nomicai-modernbert-embed-base"
                        "devstral-2-123b-instruct-2512"
                        "mistralai/devstral-small-2-2512"
                        "nomic-ai/text-embedding-nomic-embed-text-v1.5"
                        "qwen3-next-80b-a3b-instruct-mlx"
                        "deepswe-preview-mlx"
                        "devstral-small-2507-mlx"
                        "openai/gpt-oss-120b"
                        "google/gemma-3n-e4b"
                        "gpt-oss-20b-mlx" "google/gemma-3-4b")


Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :LIST-OK


In [13]:
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable state))
   (prog2$ 
    (if err
        (cw "~%FULL MODEL INFO ERROR: ~x0~%" err)
      (progn$
       (cw "~%FULL MODEL INFO (LM Studio v0 API):~%")
       (cw "  Total models: ~x0~%" (len models))
       (cw "  Loaded completions: ~x0~%" 
           (len (filter-loaded-completions-models models)))))
    (mv nil '(value-triple :full-ok) state))))


FULL MODEL INFO (LM Studio v0 API):
  Total models: 14
  Loaded completions: 2

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :FULL-OK


In [14]:
;; Preferences in order - first match wins
(defconst *model-prefs* '("qwen" "nemotron" "llama"))


The event ( DEFCONST *MODEL-PREFS* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *MODEL-PREFS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [15]:
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable state))
   (prog2$ 
    (if err
        (cw "~%SELECT MODEL ERROR: ~x0~%" err)
      (let ((selected (select-completions-model models *model-prefs*)))
        (if selected
            (cw "~%SELECTED MODEL: ~s0 (loaded ctx: ~x1)~%" 
                (model-info->id selected)
                (model-info->loaded-context-length selected))
          (cw "~%NO SUITABLE MODEL FOUND~%"))))
    (mv nil '(value-triple :select-ok) state))))


SELECTED MODEL: qwen/qwen3-coder-30b (loaded ctx: 80000)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :SELECT-OK


In [16]:
(defconst *test-messages*
  (list (make-chat-message :role (chat-role-user)
                           :content "What is 2+2? Answer in one word.")))


The event ( DEFCONST *TEST-MESSAGES* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *TEST-MESSAGES* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [17]:
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable err))
   (let ((selected (select-completions-model models *model-prefs*)))
     (if (not selected)
         (prog2$ (cw "~%NO MODEL TO TEST~%")
                 (mv nil '(value-triple :no-model) state))
       (mv-let (err2 response state)
         (llm-chat-completion (model-info->id selected) *test-messages* state)
         (declare (ignorable state))
         (prog2$ 
          (if err2
              (cw "~%CHAT ERROR: ~x0~%" err2)
            (cw "~%CHAT RESPONSE: ~s0~%" response))
          (mv nil '(value-triple :chat-ok) state)))))))


CHAT RESPONSE: Four

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :CHAT-OK


In [18]:
(value-triple :test-complete)

 :TEST-COMPLETE
